In [2]:
import tabulate
import time

import torch

# Deep Learning Practical Session 1

EE-559

February 24, 2021

Austin Zadoks

## Multiple views of a storage

In [91]:
# 13x13 tensor filled with ones
m = torch.full((13, 13), 1, dtype=torch.int)

# Rows of twos starting with second row, every five rows
m[1::5, :] = 2

# Columns of twos starting with second column, every five columns
m[:, 1::5] = 2

# 2x2 areas of threes at intersections of (fourth and fifth) and (ninth and tenth) rows and columns
m[3:5, 3:5] = 3
m[3:5, 8:10] = 3
m[8:10, 3:5] = 3
m[8:10, 8:10] = 3

# Nice printing to match exercise statement
print(tabulate.tabulate(m, tablefmt='plain'))

1  2  1  1  1  1  2  1  1  1  1  2  1
2  2  2  2  2  2  2  2  2  2  2  2  2
1  2  1  1  1  1  2  1  1  1  1  2  1
1  2  1  3  3  1  2  1  3  3  1  2  1
1  2  1  3  3  1  2  1  3  3  1  2  1
1  2  1  1  1  1  2  1  1  1  1  2  1
2  2  2  2  2  2  2  2  2  2  2  2  2
1  2  1  1  1  1  2  1  1  1  1  2  1
1  2  1  3  3  1  2  1  3  3  1  2  1
1  2  1  3  3  1  2  1  3  3  1  2  1
1  2  1  1  1  1  2  1  1  1  1  2  1
2  2  2  2  2  2  2  2  2  2  2  2  2
1  2  1  1  1  1  2  1  1  1  1  2  1


## Eigendecomposition

In [92]:
size = 20

diag_m = torch.diag(torch.arange(1., size + 1.))  # square diagonal matrix, note: float-type (1.) (D)
gauss_m = torch.empty((size, size)).normal_()  # square random normal matrix (M)
eigvals, _ = gauss_m.inverse().mm(diag_m).mm(gauss_m).eig()[0].sort(dim=0)  # sort(eigvals(M^-1 D M))

print(tabulate.tabulate(eigvals, headers=('Re', 'Im'), floatfmt='.4f'))

     Re      Im
-------  ------
 1.0000  0.0000
 2.0000  0.0000
 3.0000  0.0000
 4.0000  0.0000
 5.0000  0.0000
 6.0000  0.0000
 7.0000  0.0000
 8.0000  0.0000
 9.0000  0.0000
10.0000  0.0000
11.0000  0.0000
12.0000  0.0000
13.0000  0.0000
14.0000  0.0000
15.0000  0.0000
16.0000  0.0000
17.0000  0.0000
18.0000  0.0000
19.0000  0.0000
20.0000  0.0000


## Flops per second

In [40]:
size = 5000

# Generate matrices
m1 = torch.empty((size, size)).normal_()
m2 = torch.empty((size, size)).normal_()

# Record initial and final time for multiplication
t0 = time.perf_counter()
mm_result = torch.mm(m1, m2)
t1 = time.perf_counter()

# Matrix multipliciation of (A @ B) takes (A_ncol * A_nrow * B_nrow) operations
n_ops = size ** 3
flops = n_ops / (t1 - t0)

print(f'{flops:0.6e} [FLOPS] ({flops / 1e12:0.6f} [TFLOPS])')

1.565485e+11 [FLOPS] (0.156549 [TFLOPS])


## Playing with strides

In [29]:
def mul_row(m):
    """Multiply rows of a matrix by their 1-based index using a Python loop."""
    result = m.clone()
    size = m.size()
    for i in range(size[0]):
        for j in range(size[1]):
            result[i, j] *= i + 1
    return result

def mul_row_fast(m):
    """Multiply rows of a matrix by their 1-based index using PyTorch."""
    c = torch.arange(1, m.size()[0] + 1).view(-1, 1)  # a `column`; c[i] = i + 1
    return m.mul(c)

In [97]:
m = torch.full((1000, 400), 2.0)
funcs = [mul_row, mul_row_fast]
res = []

for func in funcs:
    # Time function
    t0 = time.perf_counter()
    r = func(m)
    t1 = time.perf_counter()
    # Save result
    res.append(r)
    # Check result
    if not torch.all(r == res[0]):
        raise Exception(f'`{func.__name__}`\'s result does not match the first result')
    # Print timing if result matches first result
    print(f'`{func.__name__}`: {t1 - t0:0.4e} [s]')

`mul_row`: 7.4879e+00 [s]
`mul_row_fast`: 4.5200e-04 [s]
